In [2]:
import Utils.FS.file as file
import TextPreprocess.cleaner as cleaner
from sklearn.feature_extraction.text import CountVectorizer
import Utils.misc as helper
import Utils.pandas_helper as ph
import pandas as pd

In [ ]:
eng_link_posts = file.read('eng_link_posts_2.pkl')

In [ ]:
print(eng_link_posts.shape)
eng_link_posts[['id', 'name', 'message', 'description']].sample(5)

In [ ]:
eng_link_posts = eng_link_posts[eng_link_posts['description'].notnull()]
print(eng_link_posts.shape)

In [ ]:
eng_link_posts['clean_description'] = ph.parallel_apply(eng_link_posts['description'].astype(str), cleaner.clean)

In [ ]:
eng_link_posts[['id', 'page_id', 'clean_description', 'description']].sample(10)

In [ ]:
file.write('eng_link_posts_3.pkl', eng_link_posts)

In [3]:
eng_link_posts = file.read('eng_link_posts_3.pkl')

In [4]:
eng_link_posts['page_id'] = eng_link_posts['page_id'].astype(str)

In [5]:
page_ids = eng_link_posts['page_id'].unique().tolist()

In [6]:
eng_link_posts.sort_values('page_id', inplace=True)

In [7]:
eng_link_posts.set_index('page_id', inplace=True)

In [8]:
isinstance(eng_link_posts.loc['109565802405671']['clean_description'], pd.Series)

True

In [10]:
import pandas as pd
for page_id in page_ids:
    posts = eng_link_posts.loc[page_id]['clean_description']
    if isinstance(posts, pd.Series):
        posts.to_csv('./page_text/{}.txt'.format(page_id), index=False, header=False)
    

In [11]:
import os

CORPUS_PATH = os.path.join('./', 'page_text')
filenames = sorted([os.path.join(CORPUS_PATH, fn) for fn in os.listdir(CORPUS_PATH)])

In [12]:
import numpy as np  # a conventional alias

import sklearn.feature_extraction.text as text

vectorizer = text.CountVectorizer(input='filename', stop_words='english', min_df=20)

dtm = vectorizer.fit_transform(filenames).toarray()

vocab = np.array(vectorizer.get_feature_names())

print(dtm.shape)
len(vocab)

(31054, 56165)


56165

In [ ]:
from sklearn import decomposition

num_topics = 20

num_top_words = 20

clf = decomposition.NMF(n_components=num_topics, random_state=1)


In [ ]:
doctopic = clf.fit_transform(dtm)

In [ ]:
topic_words = []

for topic in clf.components_:
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    topic_words.append([vocab[i] for i in word_idx])

In [ ]:
file.write('./page_text/doctopic-{0}-{1}.pkl'.format(num_topics, num_top_words), doctopic)

In [ ]:
doctopic = file.read('./page_text/doctopic-{0}-{1}.pkl'.format(num_topics, num_top_words))

In [ ]:
doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

In [ ]:
doctopic.shape

In [ ]:
page_ids = []

for fn in filenames:
    basename = os.path.basename(fn)
    page_id, ext = os.path.splitext(basename)
    page_ids.append(page_id)

In [ ]:
for t in range(len(topic_words)):
    print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))

In [ ]:
def pageTopic(page_id):
    i = page_ids.index(page_id)
    top_topics = np.argsort(doctopic[i,:])[::-1][0:10]
    top_topics_str = ' '.join(str(t) for t in top_topics)
    print("{}: {}".format(page_ids[i], top_topics_str)) 

pageTopic('109565802405671')

In [ ]:
def cloestPage(page_id):
    d = []
    i = page_ids.index(page_id)
    for page_id2 in page_ids:
        j = page_ids.index(page_id2)
        d.append(np.sum(np.abs(doctopic[i,:] - doctopic[j,:])))
    print(np.array(page_ids)[np.argsort(d[:])[0:10]])
    
cloestPage('109565802405671')

In [ ]:
cloestPage('5281959998')

In [ ]:
cloestPage('139188202817559')